# Browser Automation Lecture Notes
Date: 2024-07-16

We'll use this notebook to work through some examples and showcase some essential functions in Playwright.

In [1]:
import os
import random
import time

from playwright.async_api import async_playwright, expect

In [2]:
os.makedirs('data/', exist_ok=True)

In [3]:
# Start the browser
playwright = await async_playwright().start()

In [4]:
browser = await playwright.chromium.launch(headless=False)

In [5]:
await browser.close()

In [6]:
async def open_browser(headless=False):
    """
    Starts the automated browser and opens a new window
    """
    # Start playwright
    playwright = await async_playwright().start()

    # Open firefox browser, can use chromium (chrome) or others
    browser = await playwright.firefox.launch(headless=headless)
  
    # Create a new browser window
    page = await browser.new_page()

    return browser, page

In [7]:
driver, page = await open_browser()

In [8]:
# visit a URL
url = 'https://amazon.com'
await page.goto(url)

<Response url='https://www.amazon.com/' request=<Request url='https://www.amazon.com/' method='GET'>>

## XPATH
You learned about beautifulSoup to work with HTML...

Xpath is another way to navigate hierarchical structures of HTML and SVG

It's fast, versatile, and can be used in the developer console and in any computing language.

## Using XPATH
You can test xpaths in the devloper tools under the `console` tab using the function `$x()`. Read about that function [here](https://developer.chrome.com/docs/devtools/console/utilities/#xpath-function).


You can use Playwright's `locator` [function](https://playwright.dev/python/docs/locators#locate-by-css-or-xpath) to find the search box on Amazon site using an xpath or css selector. It'll return the first match unless you add `.all()` to the located elements.

Playwright allows other [locators](https://playwright.dev/python/docs/locators#quick-guide), which the developers suggest (but I don't).

Example of an xpath!

//input[@aria-label="Search Amazon"]

### Finding elements and performing actons
Browser automation is largely about locating elements on the page, and interacting with them in some way.
This can involve filling forms, mocking pressing buttons on a keyboard, clicking things.

In [9]:
# here's how you can do that with xpath
# search_bar = page.locator('//input[@aria-label="Search Amazon"]')

In [10]:
search_bar = page.get_by_placeholder("Search Amazon")

In [11]:
search_bar

<Locator frame=<Frame name= url='https://www.amazon.com/'> selector='internal:attr=[placeholder="Search Amazon"i]'>

In [12]:
await search_bar.fill('TEST')

In [13]:
search_term = 'womens shirt'
await search_bar.fill(search_term)

You can make the search either by inputting the "Enter" button, or finding the search button and clicking it.

Here's how you can press a [keyboard](https://playwright.dev/docs/api/class-keyboard) button.

In [14]:
await page.keyboard.press("Enter")

Alternatively, you can locate the button perform an [action](https://playwright.dev/python/docs/input#mouse-click) such as a mouse `click`.

In [15]:
# search_button = page.locator('//input[@id="nav-search-submit-button"]')
# await search_button.click()

### Parse the products

For each product, let's print the brand name:

In [16]:
# contains supports substring matches
xpath_product = '//div[contains(@cel_widget_id, "MAIN-SEARCH_RESULTS-")]'
product_tiles = await page.locator(xpath_product).all()
len(product_tiles)

0

Notice we're adding `.all()` to the command, this will return a list, rather than the first element.

If you run all the cells at once, the above will return zero results. This is because the browser needs to await for the element to be visible.

You can force the browser to sleep or check the element is visible.

In [17]:
# Let's wait for the first product to load
await expect(page.locator(xpath_product).first).to_be_visible()

In [18]:
# run it again, after waiting for the elements to be rendered
product_tiles = await page.locator(xpath_product).all()
len(product_tiles)

47

This is the sleeping method

In [19]:
# import asyncio

# await asyncio.sleep(2)
# product_tiles = await page.locator(xpath_product).all()

Let's parse one product

In [20]:
prod = product_tiles[0]

In [21]:
# see the text of the element
await prod.text_content()

'\n\n\n\n    \n\n\n\n    +45 colors/patternsFeatured from Amazon brandsFeatured from Amazon brands Amazon EssentialsWomen\'s Classic-Fit Short-Sleeve V-Neck T-Shirt, Multipacks  4.4 out of 5 stars 49,105  100+ bought in past month$19.00$19.00 FREE delivery Sat, Jul 20 on $35 of items shipped by AmazonOr fastest delivery Wed, Jul 17   1 sustainability feature<img alt="" src="https://m.media-amazon.com/images/I/11++B3A2NEL.png" height="24px" width="24px"/>  Sustainability featuresThis product has sustainability features recognized by trusted certifications. Safer chemicalsMade with chemicals safer for human health and the environment.As certified by<img alt="" src="https://m.media-amazon.com/images/I/51YvKwF01yL._SS200_.jpg" height="24px" width="24px"/>  OEKO-TEX STANDARD 100Learn more about OEKO-TEX STANDARD 100<img alt="" src="https://m.media-amazon.com/images/I/51YvKwF01yL._SS200_.jpg" height="36px" width="36px"/> OEKO-TEX STANDARD 100STANDARD 100 by OEKO-TEX requires every component 

Let's iterate through each product and print the brand name, which is saved as a header (h2) with a unique class in the element:

In [22]:
xpath_brand = '//h2[@class="a-size-mini s-line-clamp-1"]'
for product in product_tiles:
    brand = await product.locator(xpath_brand).text_content()
    print(brand)

Amazon Essentials
Abardsion
Artfish
Abardsion
AUTOMET
AUTOMET
AUTOMET
AUTOMET
AUTOMET
Dokotoo
Amazon Essentials
Kistore
Lunivop
OFEEFAN
MEROKEETY
MEROKEETY
AUTOMET
AUTOMET
AISEW
OFEEFAN
MEROKEETY
QINSEN
Dokotoo
MEROKEETY
MEROKEETY
AUTOMET
Blooming Jelly
Dokotoo
PRETTYGARDEN
Real Essentials
ZESICA
Dokotoo
Dokotoo
Blooming Jelly
ATHMILE
AUTOMET
Heymiss
WIHOLL
ATHMILE
Basoteeuo
Adibosy
Made By Johnny
SUNBS
Beautife
ATHMILE
SUNBS
Dokotoo


Although we did this all using Playwright, it's better to save the page source and then parse the saved results in BeautifulSoup, lxml, or whatever parsing software you prefer.

### Annotate the elements we find
Let's find all the ads, and highlight them red on the page.

In [23]:
xpath_ads = '//div[@data-asin and .//a[@aria-label="View Sponsored information or leave ad feedback"]]'
ads = await page.locator(xpath_ads).all()

In [24]:
len(ads)

11

You can "inject" attributes into elements, including style attributes.

In [25]:
elem = ads[0]

In [26]:
style = f"background-color: red !important; transition: all 0.5s linear;"

In [27]:
await elem.evaluate(f"el => el.setAttribute('style','{style}')")

In [28]:
async def stain(elem, color = 'red'):
    """
    Injects a style attribute to stain `elem` the `color` red.
    """
    style = f"background-color: {color} !important; "\
             "transition: all 0.5s linear;"
    await elem.evaluate(f"el => el.setAttribute('style','{style}')")

In [29]:
for elem in ads:
    await stain(elem)

### Get height of document

In [30]:
import pandas as pd

In [31]:
height = await page.evaluate("document.body.scrollHeight")

In [32]:
height

13503

Get the coorindates and size of each element using the `bounding_box` function.

In [33]:
await elem.bounding_box()

{'x': 516.7999877929688,
 'y': 3997.416748046875,
 'width': 250.39996337890625,
 'height': 607.199951171875}

In [34]:
ad_metadata = []
for elem in ads:
    if await elem.is_visible(): # use this function to only analyze visable elements
        rect = await elem.bounding_box()
        ad_metadata.append(rect)

In [35]:
df = pd.DataFrame(ad_metadata)

In [36]:
df['y']

0      528.716675
1      528.716675
2      528.716675
3     1778.116699
4     2172.816650
5     2801.016602
6     2801.016602
7     3409.216553
8     3409.216553
9     3997.416748
10    3997.416748
Name: y, dtype: float64

In [37]:
df['how_far_down'] = df['y'] / height

In [38]:
df.how_far_down.value_counts()

how_far_down
0.039155    3
0.207437    2
0.252478    2
0.296039    2
0.131683    1
0.160914    1
Name: count, dtype: int64

### Save receipts

In [39]:
# how to save what the emulator sees
source = await page.content()
with open('data/amazon_selenium_test.html', 'w') as f:
    f.write(source)

In [40]:
# just what's visible
screenshot = await page.screenshot(path='data/amazon_selenium_test.png')

### Parsing the results however you like
For me it means using lxml, but you can do this same thing in BeautifulSoup, and I encourage you do so...

In [41]:
from lxml import etree

In [42]:
dom = etree.HTML(open('data/amazon_selenium_test.html').read())

In [43]:
product_metadata = []
for result in dom.xpath('.//div[contains(@cel_widget_id, "MAIN-SEARCH_RESULTS")]'):
    # this is where you can parse as many fields as you like.
    brand, product_name = result.xpath('.//h2//text()')[:2]
    product_metadata.append({
        'brand': brand,
        'product_name': product_name
    })

In [44]:
pd.DataFrame(product_metadata)

,brand,product_name
0,Amazon Essentials,Women's Classic-Fit Short-Sleeve V-Neck T-Shir...
1,Abardsion,Women's Casual Basic Going Out Crop Tops Slim ...
2,Artfish,Women's Scoop Neck Sleeveless Knit Ribbed Fitt...
3,Abardsion,2024 Women Casual Activewear T Shirts Basic Cr...
4,AUTOMET,T Shirts Short Sleeve Crewneck Tees for Women ...
5,AUTOMET,Womens V Neck Tshirts Cap Sleeve Casual Tops O...
6,AUTOMET,Womens Tshirts Trendy Dressy Tops Business Cas...
7,AUTOMET,Women Tops Casual Basic T Shirts Loose Fit Cre...
8,AUTOMET,Womens Tops Casual Spring Summer Tshirts 2024 ...
9,Dokotoo,Womens Color Block Fashion Short Sleeve Crewne...


### Automate rotating "AUTOMET"
1. Find all products
2. filter to those with brand == AUTOMET
3. Find the image
    - Inject Javascript to make it spin.

In [45]:
async def spin(elem):
    """
    Injects a style attribute to rotate `elem` 180 degrees.
    """
    style = f"transform: rotate(180deg) !important; "
    await elem.evaluate(
            f"elm => elm.setAttribute('style','{style}')"
    )

Here's how to do this for ads (which we found previously)

In [46]:
for elem in ads:
    await spin(elem)

## Here's the bountry
Ultimately, I want every product image from specific brands (of y0ur choosing) to rotate continuously.
You need a full screenshot or a video is fine, too.

Also, make sure to save the results before you parse them.

In [47]:
for product in product_tiles:
    # get the brand name...
    brand_name = await product.locator(TK)
    
    # check if brand is in list
    if brand_name in brands_to_spin:
        print(product.text_content())
        # find the image TK
        spin(product)

NameError: name 'TK' is not defined

In [48]:
await driver.close()